In [1]:
import findspark
findspark.init() 

In [2]:
import msgpack
import boto3
import awswrangler as wr
from io import BytesIO
import pickle as pkl
import numpy as np
import re
import datetime

import matplotlib.pyplot as plt
import json

np.random.seed(1)

In [3]:
import pyspark
from pyspark.sql import SparkSession
import pyarrow.parquet as pq
import s3fs

spark = SparkSession.builder.appName("Dummy").master('local').config("spark.driver.memory", "60g").config('spark.dirver.maxResultSize', '6g').getOrCreate()
print("session = ", spark.getActiveSession())
config_spark_s3_adv = {'access_id': 'JLTHDJMKGTSENNZLD5YC',
                        'access_key': 'gtlSwf2xuppeomzYaAN3OFpcJiWtgAGUWNU7KIvJ',
                        'impl': 'org.apache.hadoop.fs.s3a.S3AFileSystem',
                        'endpoint': 'https://obs.ru-moscow-1.hc.sbercloud.ru',
                        'spark.driver.maxResultSize':0,
                      }

spark._jsc.hadoopConfiguration().set("fs.s3a.impl", config_spark_s3_adv["impl"])
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", config_spark_s3_adv["endpoint"])
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", config_spark_s3_adv["access_id"])
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", config_spark_s3_adv["access_key"])
spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
spark.conf.set("spark.sql.adaptive.enabled", "false")

22/12/08 10:45:21 WARN Utils: Your hostname, ecs-test288 resolves to a loopback address: 127.0.1.1; using 10.11.10.114 instead (on interface eth0)
22/12/08 10:45:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/12/08 10:45:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


session =  <pyspark.sql.session.SparkSession object at 0x7faee5acfb80>


In [4]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml import Pipeline
import pandas as pd
import numpy as np
from datetime import date
from scipy import sparse

In [5]:
ACCESS_KEY='du3wAsX0uzEBBn4AdkFpCX4ZTXvSWFVG48GA4jbT'
SECRET_KEY='WA0NJOX0AFAY4K1J92XZ'

wr.config.s3_endpoint_url = 'https://obs.ru-moscow-1.hc.sbercloud.ru'

boto3.setup_default_session(
    region_name='ru-moscow',
    aws_access_key_id=SECRET_KEY,
    aws_secret_access_key=ACCESS_KEY,
)


# Data Loading

In [6]:
def get_elements_dict(catalogue: pd.DataFrame):
    """
    Gets the result of reading json catalogue in form of pd.DataFrame.
    Returns dictionary of elements
    """
    elements = catalogue[~catalogue['elements'].isnull()]
    elements_dict = {}
    for el_id, el_features in zip(elements.index, elements['elements']):
        elements_dict[el_id] = el_features
    return elements_dict

def get_unique_features(elements_dict):
    """
    Returns all unique features that an element might have. 
    """
    unique_features = []
    for film, fields in elements_dict.items():
        unique_features.extend(list(fields.keys()))

    return set(unique_features)

In [7]:
path_catalogue = 's3://okko-prod/okko/datasets/catalogue/catalogue_00001_20221004215746.json'
catalogue = wr.s3.read_json(path_catalogue)

In [8]:
elements_dict = get_elements_dict(catalogue)
unique_features = get_unique_features(elements_dict)

### Mapping

In [6]:
path_mapping = 's3://okko-prod/pipelines/etl-catalogue-transform_0.1/202209180400/outgoing/element_mapping.par'
mapping = wr.s3.read_parquet(path_mapping)

In [7]:
mapping

,element_uid,unified_element_uid
0,e55c8ada-769c-4847-9646-3136eb90b1a8,2f810936-d747-4477-89cc-d5fa3ed4b71d
1,6150dada-0f0a-4ad9-82e4-fb6bd2ed324a,2f810936-d747-4477-89cc-d5fa3ed4b71d
2,17ea67be-8d67-4419-b32f-2bdfb787b393,2f810936-d747-4477-89cc-d5fa3ed4b71d
3,8ff747bf-d326-4b47-9d7a-b70a7d7f072e,2f810936-d747-4477-89cc-d5fa3ed4b71d
4,0b5701fc-30de-4920-9bbb-de876e714e4a,2f810936-d747-4477-89cc-d5fa3ed4b71d
...,...,...
146113,3b6c7799-0a9e-4ed1-83e3-ada02c400762,ced65093-3d2d-47c3-8567-f8b07b4a97e9
146114,14f0d8d8-07db-468c-894b-58794621c67b,bcfc9ac9-298c-4d12-8333-5024953dd805
146115,56a004e7-efdc-4dc8-a37b-33e89d3a2eb4,ddf1abdf-3f5a-4c52-af72-23bc9ecd196f
146116,c8b90cbd-8154-49ba-886b-83c1a2dde402,c5dd7543-7699-4c11-aa04-def94269fac7


In [9]:
mapping.to_csv('element_mapping.csv')

In [11]:
# mapping Dataframe to mapping dict
mapping_dict = {el_uid: unified_id for el_uid, unified_id in zip(mapping['element_uid'], mapping['unified_element_uid'])}

In [12]:
# add unified_uids to elements_dict
for element_id, element_features in elements_dict.items():
    if element_id in mapping_dict:
        print(element_features['name'])
        elements_dict[element_id]['unified_id'] = mapping_dict[element_id]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Коварные горничные. Сезон 4
Окно дикой природы. Сезон 1
Иллюзии. Сезон 1
Иллюзии. Сезон 2
Заговор против Америки. Сезон 1
Скандал. Сезон 2
Скандал. Сезон 4
Цари и пророки. Сезон 1
Чужестранка. Сезон 1
Охота: Смертельные игры. Сезон 1
Чужестранка. Сезон 3
Чужестранка. Сезон 2
Чужестранка. Сезон 4
Томми. Сезон 1
Гитлер против Америки. Сезон 1
Пытливые умы: Робототехника. Сезон 1
Человек-паук. Сезон 2
Человек-паук. Сезон 3
Пытливые умы: Наука и общество. Сезон 1
Хантик: Искатели секретов. Сезон 2
Хантик: Искатели секретов. Сезон 1
Гений. Сезон 1
Инстинкт. Сезон 1
Обломки. Сезон 1
Инстинкт. Сезон 2
Гений. Сезон 2
Уайат Сенак разрулит. Сезон 1
Их перепутали в роддоме. Сезон 1
Их перепутали в роддоме. Сезон 2
Их перепутали в роддоме. Сезон 3
Их перепутали в роддоме. Сезон 4
Их перепутали в роддоме. Сезон 5
Их перепутали в роддоме. Сезон 6
Любовница Дьявола: Унесённые страстью. Сезон 1
Бостонский Джордж. Сезон 1
Странный ангел. Сезон 1
Странный ангел. Сезон 2
Чем мы заняты в тени. Сезон 2
Дже

In [13]:
len(elements_dict)

184899

In [14]:
# join items belonging to one unified_uid
unified_items = {}

for element in elements_dict.values():
    if 'unified_id' in element:
        unified_items[element['unified_id']] = unified_items.get(element['unified_id'], [])
        unified_items[element['unified_id']].append(element)
    else:
        unified_items[element['uid']] = [element]

In [15]:
# number of unique items in catalogue
len(unified_items)

38825

In [16]:
# keep head elements only; element_uid = unified_uid
head_elements_dict = {}

for uni_uid, elements in unified_items.items():
    for element in elements:
        if element['uid'] not in mapping['element_uid']:
            head_elements_dict[uni_uid] = element
            break

### Make df from catalogue

In [17]:
# create dataframe; index = unified_uid
from tqdm import tqdm
items_df = pd.DataFrame(np.full(
                                (len(head_elements_dict), 
                                 len(unique_features)),
                                np.nan
                               ),
                        columns=list(unique_features),
                        index=list(head_elements_dict.keys()),
                        dtype='object'
                       )


for uid, features  in tqdm(head_elements_dict.items()):
    for feature_name, feature_value in  features.items():
        if feature_value == []:
            continue
        items_df.at[uid, feature_name] = feature_value

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38825/38825 [00:03<00:00, 10912.56it/s]


In [18]:
items_df.shape

(38825, 22)

In [19]:
items_df.head()

,children,age_access_type,name,availability,box_office_worldwide,director,alias,genre,average_rating,box_office_russia,...,sale_start_date,digital_uid,country,release_type,uid,age_restriction_aliases,release_year,budget,duration,actor
42c5cbc6-7a44-4f7b-a620-d86f43290d1d,NaN,12,"Энни Лейбовиц: Жизнь, увиденная через объектив",NaN,NaN,[4551ef07-32ff-43aa-92dc-35fd175479df],annie-leibovitz-life-through-a-lens,"[Documentary, 5c403894-146a-47a4-ae75-9f1956a3...",6.0,NaN,...,1301947200000,14059,[usa],RELEASE,42c5cbc6-7a44-4f7b-a620-d86f43290d1d,NaN,2006,NaN,4800000,"[6cf76651-d5e4-46db-8384-0993d3431992, 3de5e2c..."
04579553-398a-4aa9-a672-e841ed4a6380,NaN,12,Мисс Конгениальность 2: Прекрасна и опасна,NaN,NaN,[db533e1d-08c8-46b9-a3b7-997eaf2009bf],miss-congeniality-2-armed-fabulous,"[Action, Comedy, 364fdc2e-bdfe-40be-b2c5-d30f4...",6.11,NaN,...,1306612800000,18978,"[usa, australia-6400]",RELEASE,04579553-398a-4aa9-a672-e841ed4a6380,NaN,2005,45000000,6900000,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, 358e2a7..."
e785baa6-f175-42b4-9e16-4319ac7991d5,NaN,12,Звёздный путь 5: Последний рубеж,"[DTO, RENT, SUBSCRIPTION]",NaN,[cff3362f-ad91-498d-80b3-ea3ab8ec65cf],star-trek-v-the-final-frontier,"[Sci-Fi, Action, Thriller, Adventure]",6.21,NaN,...,1277323200000,2870,[usa],RELEASE,e785baa6-f175-42b4-9e16-4319ac7991d5,[CR_9_13],1989,27800000,6420000,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, 21c7a45..."
f3921ca6-7441-46cc-a83a-c6beee368bf0,NaN,12,Как раз под Рождество,NaN,NaN,[1a6c76d5-cc63-43c5-a1fe-a8720c9aced6],just-in-time-for-christmas,"[Comedy, Melodrama]",5.97,NaN,...,1526984637772,97651,"[usa, canada]",RELEASE,f3921ca6-7441-46cc-a83a-c6beee368bf0,NaN,2015,NaN,5580000,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, 39124be..."
5197752b-c863-4858-a12e-76d65c0a93e5,NaN,18,Американский психопат 2: Стопроцентная американка,NaN,NaN,[8416a09a-9127-4727-8ca2-ae04d7246016],american-psycho-ii-all-american-girl,"[Thriller, Horror]",5.61,NaN,...,1366361283109,38641,[usa],RELEASE,5197752b-c863-4858-a12e-76d65c0a93e5,NaN,2002,10000000,5280000,"[cff3362f-ad91-498d-80b3-ea3ab8ec65cf, c31972f..."


In [20]:
items_df.shape

(38825, 22)

In [21]:
items_df.isna().sum()

children                   32471
age_access_type             1919
name                           0
availability               13170
box_office_worldwide       38297
director                    1929
alias                          0
genre                       1919
average_rating              4848
box_office_russia          38524
subscription_only              0
type                           0
sale_start_date             2062
digital_uid                    0
country                     1919
release_type                   0
uid                            0
age_restriction_aliases    32073
release_year                1804
budget                     35034
duration                      99
actor                       1920
dtype: int64

In [22]:
items_df.columns

Index(['children', 'age_access_type', 'name', 'availability',
       'box_office_worldwide', 'director', 'alias', 'genre', 'average_rating',
       'box_office_russia', 'subscription_only', 'type', 'sale_start_date',
       'digital_uid', 'country', 'release_type', 'uid',
       'age_restriction_aliases', 'release_year', 'budget', 'duration',
       'actor'],
      dtype='object')

In [25]:
df = items_df[['age_access_type', 'name', 'availability',
               'director', 'alias', 'genre', 'average_rating',
               'subscription_only', 'type', 'sale_start_date',
               'digital_uid', 'country', 'release_type', 'uid',
               'release_year', 'duration', 'actor']]

In [26]:
df.isna().sum()

age_access_type       1919
name                     0
availability         13170
director              1929
alias                    0
genre                 1919
average_rating        4848
subscription_only        0
type                     0
sale_start_date       2062
digital_uid              0
country               1919
release_type             0
uid                      0
release_year          1804
duration                99
actor                 1920
dtype: int64

In [28]:
import pickle

In [29]:
with open('elements_available_KINOPOISK.pkl', 'wb') as f:
    pickle.dump(df, f)